In [1]:
# read saved results (tables) of old runs
# combine multiple runs
# make significance plots

import sys
import pandas as pd
import numpy as np
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
import pandas as pd
import core_tools as ct
import core_calib as cal
from scipy import stats


In [2]:
main_path = './Data manipulation/results'
# exp_names = ["1692118038_T10", "1692121657_T50", "1692130088_T100", "1692161204_O50", "1692172225_O100"] # , "1692107939_Z8T0"
# exp_names = ["1692351821_RFseed"]
exp_names = ["1698157809_Paper_v3_Real30", "1698169547_Paper_v3_Real30_Laplace"]
exp_comp = "_L"

In [3]:
# to combine Calib and OOB runs in one CD plot
tables_all = {}
for exp in exp_names:
    path = f"{main_path}/{exp}"
    dict_data, tables, params = ct.read_results(path)
    if len(tables_all) == 0:
        tables_all = tables
    else: # combine tables
        for table_key in tables.keys():
            col_names = np.array((tables[table_key].columns))
            col_names += exp_comp
            tables[table_key].columns = col_names
            # print(tables[table_key])
            # print(f"key {table_key} col_names", col_names)
            tables_all[table_key] = pd.concat([tables_all[table_key], tables[table_key]], axis=1)
            # print(tables_all[table_key])

# tables_all = ct.add_rank_mean(tables_all)
tables_all["brier"]

# ct.res_statistics(tables_all, params["metrics"], ".")

,RF_d,RF_opt,RF_large,Platt,ISO,Beta,CRF,VA,tlr,Rank,...,RF_opt_L,RF_large_L,Platt_L,ISO_L,Beta_L,CRF_L,VA_L,tlr_L,Rank_L,Elkan_L
Data,,,,,,,,,,,,,,,,,,,,,
cm1,0.094533,0.086007,0.087733,0.087375,0.091018,8.694636e-02,8.834332e-02,0.091493,0.097692,0.093660,...,0.085856,0.085684,0.087197,0.090606,8.707738e-02,0.087496,0.091308,0.097692,0.093660,0.217330
datatrieve,0.084615,0.081654,0.082625,0.088091,0.107152,1.080306e-01,8.022605e-02,0.103883,0.084738,0.081067,...,0.080010,0.077905,0.086588,0.101139,1.090985e-01,0.078461,0.098773,0.084738,0.081067,0.232124
kc1_class_level_defectiveornot,0.186047,0.172058,0.170711,0.177510,0.212561,1.955711e-01,1.821497e-01,0.189351,0.190726,0.221542,...,0.170893,0.168537,0.176773,0.212804,1.942270e-01,0.179705,0.188395,0.190726,0.221542,0.271796
kc1,0.109327,0.107910,0.103789,0.110347,0.112093,1.096536e-01,1.090896e-01,0.110603,0.130085,0.115971,...,0.107887,0.102887,0.110099,0.112009,1.090442e-01,0.108922,0.110472,0.130085,0.115971,0.240018
kc2,0.127002,0.117624,0.120842,0.120399,0.126052,1.212720e-01,1.185562e-01,0.121707,0.131819,0.126887,...,0.117333,0.119166,0.120073,0.125944,1.208255e-01,0.118356,0.121700,0.131819,0.126887,0.246560
kc3,0.080281,0.076856,0.075324,0.079726,0.085164,8.011226e-02,7.922819e-02,0.081873,0.085124,0.081570,...,0.075909,0.073825,0.079295,0.083251,7.927683e-02,0.078003,0.081106,0.085124,0.081570,0.185233
pc1,0.053851,0.052549,0.050539,0.055572,0.057589,5.603574e-02,5.382416e-02,0.055666,0.062280,0.056500,...,0.052759,0.049938,0.055233,0.056906,5.546432e-02,0.053263,0.055282,0.062280,0.056500,0.142905
spect,0.136914,0.123365,0.131644,0.128802,0.140677,1.304714e-01,1.254081e-01,0.133111,0.138631,0.160732,...,0.125176,0.127147,0.128215,0.140123,1.298555e-01,0.125986,0.132297,0.138631,0.160732,0.168380
spectf,0.142708,0.132627,0.127862,0.139626,0.153933,1.444128e-01,1.381310e-01,0.143342,0.146755,0.164147,...,0.131938,0.127215,0.138907,0.155685,1.436121e-01,0.141245,0.144887,0.146755,0.164147,0.172385


In [4]:

def stat_test_binary(test_table, method, metric, log=False):

    # Create two sample arrays
    Normal = np.array(test_table.iloc[:, 0])
    Modified = np.array(test_table.iloc[:, 1])

    result = Normal - Modified # for loss comparison
    # result = Modified - Normal # for acc comparison
    if metric == "acc":
        result[result > 0] = 0
        result[result < 0] = 1
    else:
        result[result > 0] = 1
        result[result < 0] = 0
    n_trials = len(result) 
    observed_successes = int(sum(result))

    if log:
        print(f"{metric} {method} observed_successes", observed_successes)

    result = stats.binomtest(observed_successes, n=n_trials, p=0.5, alternative='less')
    p_value = result.pvalue

    if log:
        print("p_value", p_value)

    alpha = 0.05

    if p_value < alpha:
        if log:
            print("Reject the null hypothesis. The observed frequency is significantly different from the expected frequency.")
        return 1
    else:
        if log:
            print("Fail to reject the null hypothesis. There is no significant difference.")
        return 0

def stat_test_ttest(test_table, method, metric, log=False):

    if metric == "acc":
        statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='less')
    else:
        statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='greater')


    if log:
        print("p_value", p_value)
    alpha = 0.05

    if p_value < alpha:
        if log:
            # print("Reject the null hypothesis. The observed frequency is significantly different from the expected frequency.")
            print("The change creates significant improvment")
        return '\ding{51}'
    else:
        if metric == "acc":
            statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='greater')
        else:
            statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='less')

        if p_value < alpha:
            if log:
                print("The change creates significant decline.")
            return '\ding{55}'
        else:
            if log:    
                print("Fail to reject the null hypothesis. There is no significant difference.")
        return "  "



In [5]:
calib_methods = [   "RF_d", "RF_opt", "RF_large",
                    "Platt", "ISO", "Beta", "CRF", "VA",
                    "tlr", "Rank", "Elkan",                      
                ]

metrics =  ["acc", "brier", "logloss", "ece"]

res_table = []
for method in calib_methods:
    res_array = [method]
    for metric in metrics:
        test_table = tables_all[metric][[method, method + exp_comp]]

        # if method == "RF_opt" and metric == "brier":
        #     print(test_table)

        res_array.append(stat_test_ttest(test_table, method, metric))
    res_table.append(res_array)

res_table = np.array(res_table)

res_table = pd.DataFrame(res_table, columns=["calib_methods"]+ metrics)
res_table.set_index('calib_methods', inplace=True)
res_table.to_latex(f"./comp_stat_test.txt")
res_table

/var/folders/gm/nlbfzh25075c0jwn9khghrfw0000gn/T/ipykernel_95495/453822940.py:24: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  res_table.to_latex(f"./comp_stat_test.txt")


,acc,brier,logloss,ece
calib_methods,,,,
RF_d,\ding{51},\ding{51},\ding{51},\ding{55}
RF_opt,,\ding{55},\ding{51},\ding{51}
RF_large,,,\ding{51},\ding{51}
Platt,,\ding{51},\ding{51},
ISO,,,,
Beta,\ding{51},\ding{51},\ding{51},
CRF,,,\ding{51},\ding{51}
VA,,,,
tlr,,,,
